In [1]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import MessagesState
from langgraph.graph import START,END, StateGraph
from langchain_cohere import ChatCohere
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.prompts.chat import HumanMessagePromptTemplate
import os
from dotenv import load_dotenv
import regex as re
from resources import cv, jd

In [2]:
load_dotenv()
key = os.getenv("COHERE_API_KEY")

llm = ChatCohere(cohere_api_key=key, temperature= 0.1)

In [3]:
class state(MessagesState):
    cv : str
    job_description : str
    keywords : str
    score : int
    suggestions : str

In [4]:
attempts = 0
max_attempts = 3

generater_messages = ChatPromptTemplate.from_messages([
   ("system", 
    """ You are an excellent CV generating AI assistant. Given a base CV and the Job description,
        your job is to generate a CV that perfectly matches the job description. Especially it should reflect
        in the candidate's summary section, skills and expertise section, experience and the type of projects
        they have worked on. The CV should contain all the ATS friendly keywords so that it can easily pass the 
        Application tracking system. Don't alter the template of the base CV. Don't add any additional information
        that is not present in the base CV. Write the content that is already present in a different way by
        including all the necessary keywords.
                  
        Some times the user might provide a list of keywords to be used and some suggestions to improve their CV.
        Keep that as well into account while generating the CV.
        
        Your response formate should be : "cv : [CV]" """
    ),
    MessagesPlaceholder("history")
])

generater_human_message = ChatPromptTemplate.from_messages(
    [
    ("human",
    """ Here's my base CV: {base_cv}.
    And this is my job description: {job_description}.

    Add these keywords to the CV: {keywords}.
    Here are some suggestions: {suggestions}.""")
    ]
    
    )

evaluater_messages = ChatPromptTemplate.from_messages([(
    "system",
    """ You are an excellent CV evaluating AI assistant. Given a CV and a job description, your
        job is to evaluate how well the CV is matching with the job description. You have to check whether the
        CV contains all the ATS friendly keywords. You have to provide a score out of 100 based on these
        mentioned criterias. Also if you think that the CV needs any improvement you need to provide
        area of improvements in not more than 200 words.

        Your response formate should be : 
        
        "score : [score]. 
        suggestions : [suggestions].
        missing keywords : [keywords]." """),

        MessagesPlaceholder("history")
])

evaluater_human_message = ChatPromptTemplate.from_messages([
    ("human", 
     """This is my CV: {cv}. This is my job description: {job_description}.""")])


              

In [5]:
def generater(state : state):
    global generater_human_message,generater_messages
    print("*********************INSIDE GENERATER*********************")
    generater_human_prompt = generater_human_message.invoke({"base_cv": state["cv"], "job_description": state["job_description"], "keywords": state["keywords"], "suggestions": state["suggestions"]})
    state["messages"] = generater_human_prompt.messages
    gprompt = generater_messages.invoke({"history": state["messages"]})
    print(gprompt)
    response = llm.invoke(gprompt)
    pattern = r"cv : ((.|\n)+)"
    print(response.content[:500])
    match = re.search(pattern, response.content.lower())
    if match:
        print("================================================MATCHED!!!!!!!!!!!===============================")
        state["cv"] = match.group(1)
    return {"cv" : state["cv"]}

def evaluater(state : state):
    global evaluater_human_message, evaluater_messages
    print("*********************INSIDE EVALUATOR*********************")
    print("cv : ", state["cv"][:500])
    evaluater_human_prompt = evaluater_human_message.invoke({"cv": state["cv"], "job_description": state["job_description"]})
    state["messages"] = evaluater_human_prompt.messages
    eprompt = evaluater_messages.invoke({"history": state["messages"]})
    response = llm.invoke(eprompt)
    pattern = r"score.*?(\d+)"
    match = re.search(pattern, response.content.lower())
    if match:
        state["score"] = int(match.group(1))
    pattern = r"(?<=suggestions:\n)((?:- .*\n?)+)"
    match = re.search(pattern, response.content.lower())
    if match:
        state["suggestions"] = match.group(1)
    pattern = r"(?<=missing keywords:\n)((?:- .*\n?)+)"
    match = re.search(pattern, response.content.lower())
    if match:
        state["keywords"] = match.group(1)

    return {"messages" : response, "score" : int(state["score"]), "keywords" : state["keywords"], "suggestions" : state["suggestions"]}

def decide_node(state: state):
    global attempts, max_attempts
    attempts += 1
    print("================================ATTEMPT : ", str(attempts), "================================")
    score = state["score"]
    print("*********************INSIDE DECIDE NODE*********************")
    print("Score : ", score)
    if score >= 90 or attempts == max_attempts:
        print(state["cv"])
        return END
    else:
        return "generater"

In [6]:
graph = StateGraph(state)

graph.add_node("generater", generater)
graph.add_node("evaluater", evaluater)
graph.add_edge(START, "generater")
graph.add_edge("generater", "evaluater")
graph.add_conditional_edges("evaluater", decide_node)

compiled_graph = graph.compile()

In [7]:
user_input_cv = cv
user_input_jd = jd

In [8]:
print(cv)
print(jd)


DIVYAPRAKASH RATHINASABAPATHY – DATA SCIENTIST
    rdivyaprakash78@gmail.com|| +44 7818337189 ||https://www.linkedin.com/in/divyaprakash-rathinasabapathy-6340861a7/
London, UK.
Passionate Data Scientist with hands-on experience in developing and implementing machine learning models and AI-driven solutions to address real-world challenges. Proficient in leveraging Python for data manipulation and analysis of complex datasets, with a focus on unstructured data, including text, images, and videos. Demonstrated expertise in applying statistical and machine learning techniques to derive actionable insights, communicate technical solutions to non-technical stakeholders, and contribute to data-driven regulatory practices. Seeking to apply my skills to help the ASA effectively monitor online advertising compliance and enhance consumer protection. Portfolio Link.

EDUCATION	
Data Science, M.Sc., – Kingston University, U.K.			                                                                     

In [9]:
messages = compiled_graph.invoke({"messages": "", "cv": cv, "job_description": jd, "keywords": "","suggestions" : "", "score": 0})
for m in messages['messages']:
    m.pretty_print()

*********************INSIDE GENERATER*********************
messages=[SystemMessage(content=' You are an excellent CV generating AI assistant. Given a base CV and the Job description,\n        your job is to generate a CV that perfectly matches the job description. Especially it should reflect\n        in the candidate\'s summary section, skills and expertise section, experience and the type of projects\n        they have worked on. The CV should contain all the ATS friendly keywords so that it can easily pass the \n        Application tracking system. Don\'t alter the template of the base CV. Don\'t add any additional information\n        that is not present in the base CV. Write the content that is already present in a different way by\n        including all the necessary keywords.\n                  \n        Some times the user might provide a list of keywords to be used and some suggestions to improve their CV.\n        Keep that as well into account while generating the CV.\n     